In [1]:
from func_aux import *

In [2]:
#O primeiro é o 0 e o segundo é o 1 (para jogos de dois jogadores).
def linearPond(estado,jogador,pesos,funcoes,win):
    """Função que pega no estado e no jogador e se fim do jogo devolve win*utilidade (* 1 ou -1, dependendo do jogador)
    senão devolve a combinação linear de pesos e features."""
    if estado.is_game_over():
        aux = estado.result()
        return aux*win if jogador == 0 else aux*-win
    return sum([p*f(estado,jogador) for (p,f) in zip(pesos,funcoes)])


class JogadorLinearPond(JogadorAlfaBeta):
    """SubClasse dos jogadores alfabeta que usa o combinador linear como função de avaliação.
    Recebe para além do nome e da profundidade, a lista de pesos e das funções-features, criando fun_eval"""
    def __init__(self, nome, depth,weights,features,win_value):
        self.nome = nome
        fun_eval=lambda estado,jogador: linearPond(estado,jogador,weights,features,win_value)
        self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)


In [3]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44_otimo(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [4]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44_default(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 1
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 1
    PITS_WEIGHT = 1

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [5]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def maximumVulnerableSeeds(state, player):
    opponent = 1 - player
    player_pits = state.state[:state.SOUTH_SCORE_PIT] if player == 0 else state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT]
    opponent_pits = state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT] if player == 0 else state.state[:state.SOUTH_SCORE_PIT]
    max_vulnerable_seeds = 0
    for i in range(min(len(player_pits), len(opponent_pits))):
        if player_pits[i] == 0:
            seeds_in_opponent_pit = opponent_pits[i]
            if seeds_in_opponent_pit > max_vulnerable_seeds:
                max_vulnerable_seeds = seeds_in_opponent_pit
    return max_vulnerable_seeds

def ownOverHalfOfStones(state, player):
    total_stones = sum(state.state)
    player_stones = state.state[:state.SOUTH_SCORE_PIT] if player == 0 else state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT]
    return sum(player_stones) > total_stones / 2

def ownOverHalfOfStonesOP(state, player):
    total_stones = sum(state.state)
    opponent_stones = state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT] if player == 0 else state.state[:state.SOUTH_SCORE_PIT]
    return sum(opponent_stones) > total_stones / 2

def extraTurnsScore(state, ourSide):
    our_pit = state.SOUTH if ourSide == 0 else state.NORTH
    extra_turns = state.state[our_pit]
    return extra_turns

def clusterTowardsStore(state, ourSide):
    our_pit = state.SOUTH if ourSide == 0 else state.NORTH
    opponent_pit = 1 - our_pit
    our_cluster = 0
    opponent_cluster = 0
    for i in range(our_pit, our_pit + 6):
        our_cluster += state.state[i]
    for i in range(opponent_pit, opponent_pit + 6):
        opponent_cluster += state.state[i]
    cluster_difference = our_cluster - opponent_cluster
    return cluster_difference

def func_44_teste(estado, jogador):
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3
    OWN_OVER_HALF_WEIGHT = 10
    EXTRA_TURNS_WEIGHT = 5
    CLUSTER_WEIGHT = 2

    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador) + maximumVulnerableSeeds(estado, jogador)
        + EXTRA_TURNS_WEIGHT * extraTurnsScore(estado, jogador) + CLUSTER_WEIGHT * clusterTowardsStore(estado, jogador)
    )
    if ownOverHalfOfStones(estado, jogador):
        evaluation += OWN_OVER_HALF_WEIGHT
        
    if ownOverHalfOfStonesOP(estado, jogador):
        evaluation -= OWN_OVER_HALF_WEIGHT

    return evaluation

In [6]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score, opponent_score = estado.state[estado.SOUTH_SCORE_PIT], estado.state[estado.NORTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = sum(min(pit, 6) for pit in player_pits)
    opponent_pit_values = sum(min(pit, 6) for pit in opponent_pits)
    
    return opponent_pit_values - player_pit_values

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = sum((sum(state.state[:state.SOUTH_SCORE_PIT]) - sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])) * 0.05 for state in next_states)
    
    return opponent_capture_penalty

def func_44_gpt(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT, SEEDS_WEIGHT, PROGRESS_WEIGHT, PITS_WEIGHT = 4, 1, 2, 3

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [7]:
def calculate_score_difference(estado, jogador):
    player_score, opponent_score = estado.state[estado.SOUTH_SCORE_PIT], estado.state[estado.NORTH_SCORE_PIT]
    return player_score - opponent_score if jogador == 0 else opponent_score - player_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score, opponent_score = estado.state[estado.SOUTH_SCORE_PIT], estado.state[estado.NORTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * abs(player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = sum(min(pit, 6) for pit in player_pits)
    opponent_pit_values = sum(min(pit, 6) for pit in opponent_pits)
    
    return opponent_pit_values - player_pit_values

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = sum((sum(state.state[:state.SOUTH_SCORE_PIT]) - sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])) * 0.05 for state in next_states)
    
    return opponent_capture_penalty

def func_44_gpt123(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT, SEEDS_WEIGHT, PROGRESS_WEIGHT, PITS_WEIGHT = 4, 1, 2, 3

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [8]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT]
    return player_score - opponent_score if jogador == 0 else opponent_score - player_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits) if jogador == 0 else sum(opponent_pits) - sum(player_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds if jogador == 0 else 2 * (opponent_score - player_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values) if jogador == 0 else sum(player_pit_values) - sum(opponent_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += abs(south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3
    CAPTURE_PENALTY_WEIGHT = 2.5 # Introduce a weight for capture penalty

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        CAPTURE_PENALTY_WEIGHT * calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

#  Testes entre Bonecos

In [30]:
def chapiteau_algorithm(estado, jogador):
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

In [10]:
j = 0
i = 3
while j <= 2:
    while i <= 8:
        otimo = JogadorAlfaBeta('Campeão', i, func_44_otimo)
        teste3 = JogadorAlfaBeta('AFK', i, func_44)
        teste2 = JogadorAlfaBeta('Cascao', i, func_44_gpt)
        teste1 = JogadorAlfaBeta('Estrubilho', i, func_44_gpt123)
        aux = [otimo, teste1, teste2, teste3]
        print(torneio(50, aux))
        print("")
        i += 1
    j += 1

{'Campeão': 445, 'Estrubilho': 442, 'Cascao': 453, 'AFK': 420}

{'Campeão': 472, 'Estrubilho': 510, 'Cascao': 431, 'AFK': 356}

{'Campeão': 425, 'Estrubilho': 441, 'Cascao': 444, 'AFK': 458}

{'Campeão': 450, 'Estrubilho': 475, 'Cascao': 432, 'AFK': 398}



In [ ]:
1868 17921